# Метод работы по спектрограмме
Отрезок аудиозаписи должен быть продолжительностью не менее 10 секунд 

In [60]:
# system packages
import os
import cv2
import time
import numpy as np
import shutil
import librosa.display
from tqdm import tqdm
import gc
import pandas as pd
import copy
import math
from sklearn.metrics import confusion_matrix, classification_report
from pydub import AudioSegment
import random
# torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset,RandomSampler
import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder


import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

In [61]:
test_transforms = T.Compose([T.ToTensor()])
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.cuda.empty_cache()


def infer(model,wav_file,window_size=1024, transforms= None):
    '''
    
    '''
    with torch.no_grad():
        # раздел создания спектрограммы для классификации должен быть полностью 
        # идентичен разделу создания спектрограмм для тренировки. главное параметры n_fft=window_size, hop_length=512
        #Audio = AudioSegment.from_wav(wav_file) # считываем аудио
        #n=20
        #sample = Audio[n*1000:(n+10)*1000]
        #sample_name = './temp.wav'
        #sample.export(sample_name, format="wav")
        
        
        y, sr = librosa.load(wav_file)
        window = np.hanning(window_size)
        stft  = librosa.core.spectrum.stft(y, n_fft=window_size, hop_length=512, window=window)
        out = 2 * np.abs(stft) / np.sum(window)
        fig = plt.Figure(frameon=False)
        canvas = FigureCanvas(fig)
        ax = fig.add_subplot(111)
        ax.axis('off')
        p = librosa.display.specshow(librosa.amplitude_to_db(out, ref=np.max), ax=ax)

        # спктрограмма сохраняется во временный файл
        temp = wav_file.split('/')[-1]
        temp = r'./' + temp.split('.')[0]+'.png'

        fig.savefig(temp
                    ,pad_inches = 0)


        # считываем так же как в классе Dataset
        image = cv2.imread(temp, cv2.IMREAD_COLOR)
        os.remove(temp)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0


        image_tensor = test_transforms(image).float()
        image_tensor = image_tensor.unsqueeze_(0)
        image_tensor.to(device)

        # осуществляем предсказание
        output = model(image_tensor)
        # Я в качеству выходного слоя использую простой полносвязный слой потому переменная output  
        # выглядит следующим образом tensor([[ 1.1148, -5.1110,  0.7156, -3.0015, -6.2952, -7.0076, -4.0860, -2.2178,
        #     -3.6325, -2.1464,  4.4616,  0.0611]]). Я считаю что такой подход лучше всего 
        # потому что не связывает руки
        index = torch.argmax(output, dim=1)
        # В переменной index лежит индекс самого большого значения в тензоре. для примера выше это будет 10.
        # Если нужно то тензор output можно переделать в тензор с вероятностями командами  
        # sm = torch.nn.Softmax()
        # probabilities = sm(b)
        # тогда тензор с вероятностями для примера выше будет выглядеть следующим образом
        # tensor([[3.2745e-02, 6.4759e-05, 2.1967e-02, 5.3391e-04, 1.9815e-05, 9.7192e-06,
        #     1.8049e-04, 1.1690e-03, 2.8406e-04, 1.2556e-03, 9.3035e-01, 1.1417e-02]],
        # и самая большая вероятность будет опять у элемента с индексом 10
        return index, output

In [62]:
def return_model(num_classes):
    '''
    Эта функция возвращает экземпляр модели, построенной под наше количество классов.
    Мы берем предобученную сеть, например mobilenet_v3_large и меняем ей "голову"
    '''
    model = models.mobilenet_v3_large(pretrained = True)
    
    #print(model)
    
    last_layer_input_features = model.classifier[-1].in_features
    
    model.classifier[-1] = nn.Linear(last_layer_input_features, num_classes, bias = True)
    
    #print(model)
    
    return model

In [63]:
sm = torch.nn.Softmax()

In [64]:
# путь к модели
PATH = './best_spectrogram_12classes_10_noise.pt'

In [65]:
model1=return_model(12) # создаем нашу модель
model1.load_state_dict(torch.load(PATH))
model1.eval()

MobileNetV3(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): ConvNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        )
      )
    )
    (2): InvertedResidual(
      (block): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False

In [66]:
int_to_class={0: 'Исправный, заводское состояние, новый',
              1: 'Дефект наружнегл кольца, средний',
              2: 'Дефект наружнего кольца, крупный',
              3: 'Дефект внутреннего кольца, средний',
              4: 'Дефект внутреннего кольца, крупный',
              5: 'Дисбаланс 15 гр',
              6: 'Дисбаланс 30 гр',
              7: 'Промыт полностью от смазки. Отсутсивие смазки',
              8: 'Недостаток смазки',
              9: 'Грязь в смазке',
              10: 'Сильно загрязнённая смазка',
              11: 'Дефект наружнегл кольца, лёгкий'}

In [91]:
# запуск модели иполучение ответов
index,tensor=infer(model1,r'Зашумленные датасеты/10 процентов/split10/Test5-10/Test5-10_7_.wav')  
print(f'пример выходного тензора {tensor}')
print(f'вероятности классов {sm(tensor)}')
print(f'какой индекс получили из тензора {index}')
print(f'сопоставление индекса и имени класса {int_to_class[index.numpy()[0]]}')

пример выходного тензора tensor([[ -2.0225,  -6.2516,  -1.0000,  -7.9442,  -8.4857,   1.8636,   1.0736,
          -2.0915,   2.2668,  -2.9575,  -6.8392, -14.5490]])
вероятности классов tensor([[6.7172e-03, 9.7834e-05, 1.8674e-02, 1.8006e-05, 1.0478e-05, 3.2725e-01,
         1.4852e-01, 6.2694e-03, 4.8976e-01, 2.6369e-03, 5.4363e-05, 2.4377e-08]])
какой индекс получили из тензора tensor([8])
сопоставление индекса и имени класса Недостаток смазки


C:\Users\Ysiberia\AppData\Local\Temp\ipykernel_17080\3775193975.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(f'вероятности классов {sm(tensor)}')


# Метод работы по векторному представлению

In [1]:
# system packages
import os
import librosa
import numpy as np
import librosa.display
import time
import scipy
from scipy import stats
import pandas as pd
import tqdm
import math 

# ml packages
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# torch packages
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# visualisation
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def get_base_features(wav_path:str,num_mfcc:int):
    '''
    данная функция возвращает характеристики аудиофайла в виде вектора 
    и список названий характеристик
    
    ==Input==
    wav_path - путь в аудиофайлу
    num_mfcc - количество мел
    '''
    ff_list = [] # list values
    ff_name_list=[] # list names
    y, sr = librosa.load(wav_path, sr=None)
 
    y_harmonic, y_percussive = librosa.effects.hpss(y) #Decompose an audio time series into harmonic and percussive components.
 
    tempo, beat_frames = librosa.beat.beat_track(y=y_harmonic, sr=sr)
    chroma = librosa.feature.chroma_cens(y=y_harmonic, sr=sr)
    mfccs = librosa.feature.mfcc(y=y_harmonic, sr=sr, n_mfcc=num_mfcc)
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    contrast = librosa.feature.spectral_contrast(y=y_harmonic, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zrate = librosa.feature.zero_crossing_rate(y_harmonic)
 
    chroma_mean = np.mean(chroma, axis=1)
    chroma_std = np.std(chroma, axis=1)
 
    for i in range(0, len(chroma_mean)):
        ff_list.append(chroma_mean[i])
        ff_name_list.append(f'chroma_mean_{i}')
    for i in range(0, len(chroma_std)):
        ff_list.append(chroma_std[i])
        ff_name_list.append(f'chroma_std_{i}')
    mfccs_mean = np.mean(mfccs, axis=1)
    mfccs_std = np.std(mfccs, axis=1)
 
    for i in range(0, len(mfccs_mean)):
        ff_list.append(mfccs_mean[i])
        ff_name_list.append(f'mfccs_mean_{i}')
    for i in range(0, len(mfccs_std)):
        ff_list.append(mfccs_std[i])
        ff_name_list.append(f'mfccs_std_{i}')         
    cent_mean = np.mean(cent)
    cent_std = np.std(cent)
    cent_skew = scipy.stats.skew(cent, axis=1)[0]

 
    contrast_mean = np.mean(contrast,axis=1)
    contrast_std = np.std(contrast,axis=1)
 
    rolloff_mean=np.mean(rolloff)
    rolloff_std=np.std(rolloff)

    data = np.concatenate(([cent_mean, cent_std, cent_skew], 
                           contrast_mean, contrast_std, 
                           [rolloff_mean, rolloff_std, rolloff_std]), axis=0)
    ff_list += list(data)
    ff_name_list+=['cent_mean', 'cent_std', 'cent_skew']
    for i in range(0,len(contrast_mean)):
        ff_name_list.append(f'contrast_mean_{i}')
    for i in range(0,len(contrast_std)):
        ff_name_list.append(f'contrast_std_{i}')
    ff_name_list+=['rolloff_mean', 'rolloff_std', 'rolloff_std']
    
    
    zrate_mean = np.mean(zrate)
    zrate_std = np.std(zrate)
    zrate_skew = scipy.stats.skew(zrate,axis=1)[0]
 
    ff_list += [zrate_mean, zrate_std, zrate_skew]
    ff_name_list+=['zrate_mean', 'zrate_std', 'zrate_skew']
    ff_list.append(tempo)
    ff_name_list.append('tempo')
 
    return ff_list,ff_name_list

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 1024)
        self.layer_2 = nn.Linear(1024, 512)
        self.layer_3 = nn.Linear(512, 256)
        self.layer_4 = nn.Linear(256, 128)
        self.layer_5 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(1024)
        self.batchnorm2 = nn.BatchNorm1d(512)
        self.batchnorm3 = nn.BatchNorm1d(256)
        self.batchnorm4 = nn.BatchNorm1d(128)
        self.batchnorm5 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_4(x)
        x = self.batchnorm4(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_5(x)
        x = self.batchnorm5(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [5]:
NUM_FEATURES=72
NUM_CLASSES = 11

In [12]:
sm = torch.nn.Softmax()

In [6]:
best_weights = './best_vector_12_classes_10_noise_new.pt' # куда сохраняем веса с лучшим результатом

In [7]:
# сначала надо создать объект нейронной сети, он будет как бы пустой
model = MulticlassClassification(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
# загрузить в него веса этой же но обученной сети
# если архитектура "пустышки" будет отличаться от архитектуры которую загружаем то будет ошибка
model.load_state_dict(torch.load(best_weights)) # веса взяты из обучения
model.eval() # включаем режим предсказания (отключаем обратное распространение ошибки)
model.to(device) # отправляем на устройство

MulticlassClassification(
  (layer_1): Linear(in_features=72, out_features=1024, bias=True)
  (layer_2): Linear(in_features=1024, out_features=512, bias=True)
  (layer_3): Linear(in_features=512, out_features=256, bias=True)
  (layer_4): Linear(in_features=256, out_features=128, bias=True)
  (layer_5): Linear(in_features=128, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=11, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (batchnorm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [8]:
int_to_class={0: 'Исправный, заводское состояние, новый',
              1: 'Дефект наружнегл кольца, средний',
              2: 'Дефект наружнего кольца, крупный',
              3: 'Дефект внутреннего кольца, средний',
              4: 'Дефект внутреннего кольца, крупный',
              5: 'Дисбаланс 15 гр',
              6: 'Промыт полностью от смазки. Отсутсивие смазки',
              7: 'Недостаток смазки',
              8: 'Грязь в смазке',
              9: 'Сильно загрязнённая смазка',
              10: 'Дефект наружнегл кольца, лёгкий'}

In [9]:
def infer(model,file_path,num_mfcc,windows_size = 1024):
    '''
    Векторизуем аудиофайл и отправляем в сеть. Тут все параметры векторизации должны совпадать 
    с параметрами векторизации которые были при создании датасета для обучения. 
    чтобы размерность векторов совпадала
    ===Input===
    model - модель которая будет осуществлять предсказание
    file_path - путь в аудиофайлу
    num_mfcc - количество мел-кепстров которые будет доставать из аудио
    
    '''
    with torch.no_grad():
        vector, names = get_base_features(file_path,num_mfcc)
        vector = torch.FloatTensor(vector)
        vector = vector.unsqueeze_(0)
        vector.to(device)

         # осуществляем предсказание
        output = model(vector)
        # Я в качеству выходного слоя использую простой полносвязный слой потому переменная output  
        # выглядит следующим образом (tensor([[ 7.3186, -2.7376, -3.3811, -2.7901, -4.9589, -2.8915, -2.0703, -4.0185,
        #  -3.1207, -3.0112, -1.7625, -4.7538]]). Я считаю что такой подход лучше всего 
        # потому что не связывает руки
        index = torch.argmax(output, dim=1)
        # В переменной index лежит индекс самого большого значения в тензоре. для примера выше это будет 10.
        # Если нужно то тензор output можно переделать в тензор с вероятностями командами  
        # sm = torch.nn.Softmax()
        # probabilities = sm(b)
        # тогда тензор с вероятностями для примера выше будет выглядеть следующим образом
        # tensor([[9.9958e-01, 4.2901e-05, 2.2542e-05, 4.0708e-05, 4.6534e-06, 3.6782e-05,
        # 8.3619e-05, 1.1917e-05, 2.9250e-05, 3.2634e-05, 1.1375e-04, 5.7129e-06]]),
        # и самая большая вероятность будет опять у элемента с индексом 0
        return output,index

In [27]:
tensor,index = infer(model,r'C:\Users\Ysiberia\Documents\GitHub\audiio_classification\Зашумленные датасеты\10 процентов\split10\Test8-10\Test8-10_5_.wav',12)
print(f'пример выходного тензора {tensor}')
print(f'вероятности классов {sm(tensor)}')
print(f'какой индекс получили из тензора {index}')
print(f'сопоставление индекса и имени класса {int_to_class[index.numpy()[0]]}')

пример выходного тензора tensor([[ 20.1238,   2.5616, -33.6511,  -8.0080, -14.4073, -13.1352,   8.8881,
         -22.0474, -22.6461, -21.0331, -18.7734]])
вероятности классов tensor([[9.9999e-01, 2.3594e-08, 4.4241e-24, 6.0604e-13, 1.0077e-15, 3.5957e-15,
         1.3194e-05, 4.8447e-19, 2.6623e-19, 1.3359e-18, 1.2798e-17]])
какой индекс получили из тензора tensor([0])
сопоставление индекса и имени класса Исправный, заводское состояние, новый


C:\Users\Ysiberia\AppData\Local\Temp\ipykernel_19308\3897188829.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(f'вероятности классов {sm(tensor)}')
